In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df = pd.read_csv(os.path.join("train.csv","train.csv"))

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
from tensorflow.keras.layers import TextVectorization

In [5]:
df[df.columns[2:]]

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [6]:
df['comment_text']

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [7]:
x = df['comment_text']
y = df[df.columns[2:]].values

In [8]:
MAX_FEATURES = 50000 #NUM OF WORDS IN VOCAB

In [9]:
vectorizer = TextVectorization(max_tokens = MAX_FEATURES , output_sequence_length =1000 , output_mode = 'int')

In [10]:
vectorizer.adapt(x.values)

In [11]:
vectorized_text = vectorizer(x.values)

In [12]:
vectorized_text

<tf.Tensor: shape=(159571, 1000), dtype=int64, numpy=
array([[  643,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2506, ...,     0,     0,     0],
       [  425,   440,    70, ...,     0,     0,     0],
       ...,
       [32141,  7329,   383, ...,     0,     0,     0],
       [    5,    12,   533, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [13]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file dl pipeline 
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(40000)
dataset = dataset.batch(8)
dataset = dataset.prefetch(4)

In [14]:
dataset.as_numpy_iterator().next()

(array([[    2,   113,   453, ...,     0,     0,     0],
        [   42,  5257,    20, ...,     0,     0,     0],
        [   94,    13,  2574, ...,     0,     0,     0],
        ...,
        [    8,    55,    69, ...,     0,     0,     0],
        [   38,   393,  4433, ...,     0,     0,     0],
        [40530,    28,  2931, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [15]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [18]:
train.as_numpy_iterator().next()

(array([[  52,    2,   63, ...,    0,    0,    0],
        [ 227,  191,   22, ...,    0,    0,    0],
        [   1,   46,  291, ...,    0,    0,    0],
        ...,
        [  48,  155, 3333, ...,    0,    0,    0],
        [  14,    9,   38, ...,    0,    0,    0],
        [  12,  183,  119, ...,    0,    0,    0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Bidirectional , Dense , Embedding , Dropout

In [22]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1 , 32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128 , activation='relu'))
model.add(Dense(6 , activation='sigmoid'))

In [27]:
model.compile(loss='BinaryCrossentropy', optimizer='adam')

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          1600032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [29]:
history = model.fit(train , epochs=1 , validation_data=val)

13962/13962 [==============================] - 4837s 346ms/step - loss: 0.0618 - val_loss: 0.0483


In [31]:
history.history

{'loss': [0.06181428208947182], 'val_loss': [0.048312265425920486]}

In [35]:
batch_X , batch_y = test.as_numpy_iterator().next()

In [33]:
inp_text = vectorizer('Fuck you!')


In [36]:
model.predict(batch_X)

1/1 [==============================] - 0s 74ms/step


array([[1.0684817e-02, 8.8124143e-06, 5.1935663e-04, 2.1534650e-05,
        1.4400869e-03, 6.1749750e-05],
       [2.0844862e-03, 3.7598920e-07, 5.6098394e-05, 7.9734014e-07,
        1.7093193e-04, 3.6966687e-06],
       [1.0901177e-03, 1.4211201e-07, 2.7171696e-05, 2.6780052e-07,
        7.9264399e-05, 1.4651706e-06],
       [2.0159620e-03, 3.5841131e-07, 5.4280623e-05, 7.6530864e-07,
        1.6432861e-04, 3.5273965e-06],
       [4.6341969e-03, 2.3058308e-06, 1.9318445e-04, 4.7487715e-06,
        5.1388989e-04, 1.7352233e-05],
       [5.8249477e-03, 3.7745717e-06, 2.7221249e-04, 7.6476827e-06,
        6.9313234e-04, 2.5868838e-05],
       [9.5401909e-03, 6.3669177e-06, 4.1659811e-04, 1.5558890e-05,
        1.2033404e-03, 4.6417928e-05],
       [2.6470970e-03, 6.3512675e-07, 7.9981161e-05, 1.3968067e-06,
        2.3773567e-04, 5.8616461e-06]], dtype=float32)

In [37]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [38]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [39]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 72ms/step


1/1 [==============================] - 0s 73ms/step


1/1 [==============================] - 0s 75ms/step


1/1 [==============================] - 0s 78ms/step


1/1 [==============================] - 0s 73ms/step


...

1/1 [==============================] - 0s 71ms/step


1/1 [==============================] - 0s 69ms/step


1/1 [==============================] - 0s 71ms/step


1/1 [==============================] - 0s 71ms/step


1/1 [==============================] - 0s 71ms/step


In [40]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.7896235585212708, Recall:0.6974015235900879, Accuracy:0.47241726517677307


In [59]:
import tensorflow as tf
import joblib

In [43]:
model.save('toxicity.h5')

In [44]:
model = tf.keras.models.load_model('toxicity.h5')

In [45]:
input_str = vectorizer('hey i freaken hate you!')


In [46]:
res = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 1s 680ms/step


In [48]:
res

In [62]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [65]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

C:\Users\suraj\anaconda3\lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\suraj\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\suraj\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


1/1 [==============================] - 0s 60ms/step


In [64]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861

Could not create share link, please check your internet connection.


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Missing argument 'TARGET'.
